In [53]:
#importing libararies
import pandas as pd
import numpy as np
import os
from __future__ import print_function
from mailmerge import MailMerge
from datetime import date
import docx
import import_ipynb
import glob
import tkinter
from tkinter import filedialog
import warnings
warnings.filterwarnings("ignore")


def format_sheet(file_path:str,template_path:str,sheet_num_template=0,run=0):
    """
    file_path: the file you want to design, must bx xlsx
    template_path: the teplate of the design, must bx xlsx
        """
    # format sheet
    xlsx=pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
    for i in range(len(xlsx.book.worksheets)):
        new_sheet=xlsx.book.worksheets[run]
        template=pd.ExcelWriter(template_path,engine='openpyxl', mode='a')
        default_sheet=template.book.worksheets[sheet_num_template]
        from copy import copy
        new_sheet.sheet_view.rightToLeft=True
        for row in default_sheet.rows:
            for cell in row:
                new_cell=new_sheet.cell(row=cell.row, column=cell.col_idx)
                if cell.has_style and new_cell.value != None:
                    new_cell.font=copy(cell.font)
                    new_cell.border=copy(cell.border)
                    new_cell.fill=copy(cell.fill)
                    new_cell.number_format=copy(cell.number_format)
                    new_cell.protection=copy(cell.protection)
                    new_cell.alignment=copy(cell.alignment)
        from openpyxl.utils import get_column_letter
        for i in range(default_sheet.max_column):
            new_sheet.column_dimensions[get_column_letter(i + 1)].width=default_sheet.column_dimensions[
                get_column_letter(i + 1)].width
    
    xlsx.save()
def change_names_and_order(file_path:str,df:pd.DataFrame):
    """
    name of columns in names must be "old_name", "new_name"
        """
    match_names=pd.read_excel(file_path)
    match_names.index=match_names.old_name
    dict_names=match_names.to_dict()['new_name']
    ls_names=match_names['new_name'].tolist()
    df.rename(columns=dict_names, inplace=True)
    df=df[ls_names]
    return df

In [54]:

tkinter.Tk().withdraw() # prevents an empty tkinter window from appearing

transction_folder_path = filedialog.askdirectory()
permanant_folder_path = filedialog.askdirectory()

In [144]:

#reading the check file
filepath_check = transction_folder_path+"\*שקים*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    check = pd.read_excel(textfile)

    #reading the client file
filepath_client = transction_folder_path+"\*לקוחות*"
txt_client = glob.glob(filepath_client)
for textfile in txt_client:
    client = pd.read_excel(textfile) 

    #reading the client file
filepath_report_num = transction_folder_path+"\*מחולל*"
txt_report_num = glob.glob(filepath_report_num)
for textfile in txt_report_num:
    report_num = pd.read_excel(textfile) 

filepath_reported = transction_folder_path+"\*מעקב*"
txt_reported = glob.glob(filepath_reported)
for textfile in txt_reported:
    reported = pd.read_excel(textfile)     

fsp_list = pd.read_excel(permanant_folder_path+'\נותני שירותים פיננסים.xlsx')
rc = pd.read_excel(permanant_folder_path+'\מדינות.xlsx')
content = pd.read_excel(transction_folder_path+'\content.xlsx')


In [232]:
#drop na
check = check.dropna(subset = 'nis sum')
#check nis sum is int
check['nis sum'] = check['nis sum'].astype(int)

#stripping client names from client and templeata df
client['clients name'] = client['clients name'].str.rstrip()
client['clients name'] = client['clients name'].str.lstrip()
check['clients name'] = check['clients name'].str.rstrip()
check['clients name'] = check['clients name'].str.lstrip()

#sort the data in check by deal date and by name
check = check.dropna(subset = 'deal number')
check["deal number"] = check["deal number"].astype(int)

check = check.sort_values(['clients name','date'])

# adding the sum for every deal number and marking every deal under 50k as True
deal_sum = check.groupby('deal number').agg({'nis sum':'sum'})
check = check.merge(deal_sum,on = 'deal number' , how = 'left', suffixes = ('','_deal'))

#Adding a column stating if the transction is under or over 50K(False = over 50K)
check['U50'] = np.where(check['nis sum_deal'] >= 50000,False,True)

#Adding a column stating if the transction is under or over 5K(False = over 5K)
check['U5'] = np.where(check['nis sum_deal'] >= 5000,False,True)

#creating a column with the dates diffrence by day and creating a new column with the value as a number
check['date'] = pd.to_datetime(check['date']) 
check['date_diff'] = check['date'].diff()
check['date_diff_fl'] = check['date_diff'] / pd.to_timedelta(1, unit='D')

#testing for matching clients in match column
check['match'] = check['clients name'].eq(check['clients name'].shift())

#adding a conditional colum based on date diff values
check['diff_check'] = np.where((check['date_diff_fl'] > 3) | (check['date_diff_fl'] < 0) | (check['match'] == False) , 1 , 0)

#cummaltive sum as a way to count if the values in the diff check
check['cumsum'] = np.cumsum(check['diff_check'])

#striping blank spaces from citizenship  country
client['citizenship'] = client['citizenship'].str.rstrip()
client['citizenship'] = client['citizenship'].str.lstrip()

try:
    #id number as type int
    client['id number'] = np.where(client['id number'].isna(), 1234 , client['id number'])
    #locating plastenian autherity risk clients
    client['citizenship_ps'] = np.where((client['id number'] > 800000000) & (client['id number'] < 999999999), 'הרשות הפלשטינית' , client['citizenship'] )
    client['citizenship'] = client['citizenship_ps']
except TypeError:
    print('id number could not be as type int')

id number could not be as type int


In [233]:
#building the monitoring df with sum per client, deal distinct count and under 50k deal count
mont = check.groupby(['clients name','cumsum']).agg({'nis sum':'sum','deal number':'nunique','U50':'sum','U5':'sum'})
mont = mont.reset_index()

#sorting the values by sum
mont = mont.sort_values('nis sum',ascending = False)
mont['U50'] = mont['U50'].astype(int)
mont['U5'] = mont['U5'].astype(int)

#preform a reported in the last two month test
mont = mont.merge(reported['שם'], left_on = 'clients name' , right_on = 'שם', how = 'left')

#adding a column named status based on unique count of deal number and at least on tr under 50k and not reported in the past 2m
mont['status'] = np.where((mont['deal number']==1) | (mont['U50'] == 0) | (mont['שם'].notna()), 'Regular Report', 'Check')
mont = mont.drop_duplicates('cumsum')



#preform a financial service providers test - results stored in a column named fsp_check
#adding the id number to the monitoring sheet
mont = mont.merge(client[['id number','clients name']], on = 'clients name', how = 'left')


#adding a column מספר מזהה to the monitoring sheet values indicates fsp(finanacial service provider)
try:
    mont = mont.merge(fsp_list['מספר מזהה'], left_on = 'id number', right_on = 'מספר מזהה' , how = 'left')
except ValueError:
    mont = mont.merge(fsp_list['מספר מזהה'].astype(str), left_on = 'id number', right_on = 'מספר מזהה' , how = 'left')
mont['fsp_check'] = np.where(mont['מספר מזהה'].isna(),"check" , "financial service provider")

mont = mont.drop_duplicates('cumsum')


In [234]:
#preform a risk country test
#merging client detail with monitoring list
mont_client = mont.merge(client,on = 'clients name',how = 'left')

# a summarized data frame of clients name and country
mont_client_country = mont_client[['clients name','citizenship']]

#merging the list of risk country
mont_client_country_risk = mont_client_country.merge(rc,left_on = 'citizenship' , right_on = 'עברית' , how = 'left')

#merging the risk status for each client to the monitoring df and droping the cumsum duplicates
mont = mont.merge(mont_client_country_risk[['clients name','סיכון/ללא דיווח רגיל']],on = 'clients name', how = 'left')
mont = mont.drop_duplicates('cumsum')

In [235]:
#reported test
mont['reported'] = np.where(mont['שם'].notna(),'reported', 'check')

#marking dirdos transction as d in a new column named dirdos
mont['dirdos'] = np.where((mont['nis sum']>= 47000) & (mont['nis sum']<50000) & (mont['שם'].isna()) & (mont['fsp_check'] == 'check') & (mont['U50'] > 0) , "d",'not d')
mont['dirdos_risk'] = np.where((mont['nis sum']>= 4700) & (mont['nis sum']<5000) & (mont['שם'].isna()) & (mont['fsp_check'] == 'check') & (mont['U5'] > 0) , "dr",'not dr')


In [236]:
#montoring test export

mont_export = mont
mont_export['deal number count status'] = np.where(mont_export['deal number'] == 1 , 'הועבר דיווח רגיל','')
mont_export ['U50 status'] = np.where(mont_export['U50'] == 0 , 'הועבר דיווח רגיל','')
mont_export ['reported status'] = np.where(mont_export['reported'] == 'reported' , 'דווח בשלושת החודשים האחרונים','')
mont_export ['fsp_status'] = np.where(mont_export['fsp_check'] == 'financial service provider' , 'נש"מ','')
mont_export['nis sum status'] = np.where(mont_export['nis sum'] <= 50000 , 'סך עסקאות נמוך מרך הדיווח' , '')
        
mont_export = mont_export[['nis sum status','fsp_status','reported status','U50 status','deal number count status','clients name','nis sum','cumsum']]


        



In [153]:
#creating a new data frame contianing only clients that need to be reported
mont2r = mont[((mont['status'] == 'Check')|(mont['dirdos']=="d")) & (mont['fsp_check'] == 'check') & (mont['nis sum'] >= 47000)]
mont2r = mont2r.drop_duplicates('cumsum')

#adding the type of the report p-pitzul d-dirdos
mont2r['type'] = np.where(mont2r['nis sum'] >= 50000 , "p" , "d")

# figure out the problem of a client with multiple types of reports
type_count = mont2r[['clients name','type']].groupby('clients name').nunique()

#adding situations in which there are two types of reports for a client
mont2r = mont2r.merge(type_count,on = 'clients name', how = 'left',suffixes = ('','_count'))
mont2r['final type'] = np.where(mont2r['type_count'] > 1 , "pd" , mont2r['type'] )

#creating a new data frame with duplicate cumsum for report details df= mont2report_w_cumsum
mont2report_w_cumsum = mont2r
mont2report = mont2r.drop_duplicates('clients name')


In [154]:
#creating a new data frame contianing only clients that need to be reported for risk clients
mont2rr = mont[((mont['status'] == 'Check')|(mont['dirdos_risk']=="dr")) & (mont['U5'] != 0) & (mont['fsp_check'] == 'check') & (mont['nis sum'] >= 4700) & (mont['סיכון/ללא דיווח רגיל'].notna())]
mont2rr = mont2rr.drop_duplicates('cumsum')

#adding the type of the report pr-pitzul dr-dirdos (risk)
mont2rr['final type'] =  np.where((mont2rr['nis sum'] >= 5000), "pr" , "dr")

In [155]:
#adding the risk transctions report to the df with cumsum
mont2report_w_cumsum = mont2report_w_cumsum.append(mont2rr)

In [156]:
#sort the data in check by deal date and by owner name
check["deal number"] = check["deal number"].astype(int)
check_ow = check.sort_values(['owner name','date'])

 

#creating a column with a test for matching owner in the past transction
check_ow['match_ow'] = check_ow['owner name'].eq(check_ow['owner name'].shift())

#delete the fsp client from check_ow df
check_ow = check_ow.merge(client[['clients name','id number']], on = 'clients name' , how = 'left')


#deleting FSP from check ow:
try:
    check_ow = check_ow.merge(fsp_list['מספר מזהה'], left_on = 'id number', right_on = 'מספר מזהה', how = 'left')
except ValueError:
    check_ow = check_ow.merge(fsp_list['מספר מזהה'].astype(str), left_on = 'id number', right_on = 'מספר מזהה', how = 'left')
check_ow = check_ow[check_ow['מספר מזהה'].isna()]

#creating a column with the dates diffrence by day and creating a new column with the value as a number
check_ow['date_diff'] = check_ow['date'].diff()
check_ow['date_diff_fl'] = check_ow['date_diff'] / pd.to_timedelta(1, unit='D')

#adding a conditional colum based on date diff values
check_ow['diff_check'] = np.where((check_ow['date_diff_fl'] > 3) | (check_ow['date_diff_fl'] < 0) | (check_ow['match_ow'] == False) , 1 , 0)

#cummaltive sum as a way to count if the values in the diff check
check_ow['cumsum'] = np.cumsum(check_ow['diff_check'])



In [157]:
#building the monitoring df with sum per client and deal distinct count
mont_ow = check_ow.groupby(['owner name','cumsum']).agg({'nis sum':'sum','deal number':'nunique','clients name':'nunique','U50':'sum'})

#sorting the values by sum
mont_ow = mont_ow.sort_values('nis sum',ascending = False)

#adding a column named status based on unique count of deal number
mont_ow['status'] = np.where((mont_ow['deal number']==1) | (mont_ow['clients name'] ==1) | (mont_ow['nis sum'] <= 50000) | (mont_ow['U50'] == 0), 'Regular Report', 'Check')
mont_ow = mont_ow.reset_index()

#mont_ow for export
mont_ow_export = mont_ow
mont_ow_export['deal number count status'] = np.where(mont_ow_export['deal number'] == 1 , 'הועבר דיווח רגיל' , '')
mont_ow_export['clients name count status'] = np.where(mont_ow_export['clients name'] == 1 , 'לקוח אחד' , '')
mont_ow_export['50U status'] =  np.where(mont_ow_export['U50'] == 0 , 'אין עסקאות מתחת ל 50 אלף' , '')
mont_ow_export['nis sum status'] = np.where(mont_ow_export['nis sum'] <= 50000 , 'סך עסקאות נמוך מרך הדיווח' , '')
mont_ow_export = mont_ow_export[['deal number count status','clients name count status','50U status','nis sum status','owner name']]

#leaving only suspicious clients
mont_ow = mont_ow[mont_ow['status'] == 'Check']


In [158]:
#creating a list of the shared owner report clients name
mont_ow_merge = mont_ow.merge(check_ow[['clients name','cumsum']], on = 'cumsum', how = 'left' )

#subtracting past 3 month reported clients
mont_ow_clients = mont_ow_merge.merge (reported['שם'] , how = 'left' , left_on = 'clients name_y' , right_on = 'שם')
mont_ow_clients = mont_ow_clients[mont_ow_clients['שם'].isna()]
mont_ow_clients = mont_ow_clients.rename(columns = {'clients name_y':'clients name'})
mont_ow_clients_w_cumsum = mont_ow_clients
mont_ow_clients = mont_ow_clients.drop_duplicates('clients name') 
mont_ow_clients = mont_ow_clients.drop_duplicates('cumsum')  
mont_ow_clients = mont_ow_clients.reset_index()
mont_ow_clients['final type'] = "m"

In [159]:
#building the monitoring df with sum per client and deal distinct count
mont_ow_risk = check_ow.groupby(['owner name','cumsum']).agg({'nis sum':'sum','deal number':'nunique','clients name':'nunique','U5':'sum'})

#sorting the values by sum
mont_ow_risk = mont_ow_risk.sort_values('nis sum',ascending = False)
mont_ow_risk['status'] = np.where((mont_ow_risk['deal number']==1) | (mont_ow_risk['clients name'] ==1) | (mont_ow_risk['nis sum'] <= 5000) | (mont_ow_risk['U5'] == 0), 'Regular Report', 'Check')
mont_ow_risk = mont_ow_risk[mont_ow_risk['status'] == 'Check']

#shared owner risk tranctions
mont_ow_risk_merge = mont_ow_risk.merge(check_ow[['clients name','cumsum']], on = 'cumsum', how = 'left' )
mont_ow_risk_merge = mont_ow_risk_merge.merge(client[['clients name','citizenship']],left_on = 'clients name_y', right_on = 'clients name', how = 'left')
mont_ow_risk_merge = mont_ow_risk_merge.merge(rc[['עברית','סיכון/ללא דיווח רגיל']], left_on = 'citizenship', right_on = 'עברית', how = 'left' )
mont_ow_risk_merge = mont_ow_risk_merge[mont_ow_risk_merge['סיכון/ללא דיווח רגיל'].notna()]
mont_ow_clients_w_cumsum = mont_ow_clients_w_cumsum.append(mont_ow_risk_merge)
mont_ow_risk_merge = mont_ow_risk_merge.drop_duplicates('clients name')
mont_ow_risk_merge = mont_ow_risk_merge.drop_duplicates('cumsum')
mont_ow_risk_merge['final type_ow'] = 'mr'

In [160]:
#a new data frame containing all kinds of reports
all_clients2r = mont_ow_clients[['clients name','final type']].merge(mont2report[['clients name','final type']], on = 'clients name',how = 'outer').fillna('')
all_clients2r = all_clients2r.merge(mont2rr[['final type','clients name']], how = 'outer', on = 'clients name').fillna('')
all_clients2r = all_clients2r.merge(mont_ow_risk_merge[['final type_ow','clients name']], how = 'outer', on = 'clients name').fillna('')
all_clients2r['finaltypedown'] =  all_clients2r['final type_y'] + all_clients2r['final type_x'] + all_clients2r['final type'] + all_clients2r['final type_ow']
#all_clients2r['finaltypedown'] =  all_clients2r['final type_y'] + all_clients2r['final type_x'] 

merged = all_clients2r.drop_duplicates('clients name').reset_index(drop = True)

In [161]:
#creating the client details for reports data frame
info2r = merged[['clients name','finaltypedown']].merge(client , on = 'clients name', how = 'left').dropna(axis= 1,how = 'all' )
#try:
info2r['clients name'] = info2r['clients name'].str.replace('\d+', '')
info2r['country'] = 'ישראל'

#except KeyError:
#    print('no reports')
try:
    info2r['id number'] = info2r['id number'].fillna(0)
    info2r['id number'] = info2r['id number'].astype(int)
except:
    print('there are letters in one or more of id numbers')


In [162]:
#distinguish between company and private name
info2r['company name'] = np.where((info2r['id number'] >= 500000000) & (info2r['id number'] <= 600000000) , info2r['clients name'] ,"")
info2r['person name'] = np.where((info2r['id number'] <= 500000000) | (info2r['id number'] >= 600000000) , info2r['clients name'] ,"")


In [163]:
#distinguish between company and private dob
try:
    info2r['dob'] = (info2r['dob']).astype(str)
    info2r['company birth'] = np.where((info2r['id number'] >= 500000000) & (info2r['id number'] <= 600000000) , info2r['dob'] , '')
    info2r['person birth'] = np.where((info2r['id number'] <= 500000000) | (info2r['id number'] >= 600000000) , info2r['dob'] ,'')
except KeyError: 'dob'

In [164]:
#distinguish between company and private citizenship
info2r['company citizeship'] = np.where((info2r['id number'] >= 500000000) & (info2r['id number'] <= 600000000) , info2r['citizenship'] , '')
info2r['person citizenship'] = np.where((info2r['id number'] <= 500000000) | (info2r['id number'] >= 600000000) , info2r['citizenship'] ,'')

In [165]:
#distinguish between company and private citizenship
info2r['company id'] = np.where((info2r['id number'] >= 500000000) & (info2r['id number'] <= 600000000) , info2r['id number'] , '')
info2r['person id'] = np.where((info2r['id number'] <= 500000000) | (info2r['id number'] >= 600000000) , info2r['id number'] ,'')

In [166]:
#dropping duplicates and reseting index
info2r = info2r.drop_duplicates('clients name')
info2r = info2r.reset_index(drop = True)
#importing the report number needed

report_id = (report_num.iloc[1,5] + 1).astype(int)
info2r['report_id'] = report_id + range(len(info2r.index))
info2r['report_name'] = report_num.iloc[3,1].rsplit('-',1)[0]

In [167]:
info2r = info2r.merge(content, left_on = 'finaltypedown' , right_on = 'type', how = 'left')
mont_export = mont_export.merge(info2r[['clients name','finaltypedown']],on = 'clients name', how = 'left')
mont_export.to_excel('status.xlsx')

In [168]:
for i in info2r.index:
    info2r['Title'].iloc[i]  = info2r['Title'].iloc[i].replace('מדינה',str(info2r['citizenship'].iloc[i]) + ', מדינה')
    info2r['Content'].iloc[i]  = info2r['Content'].iloc[i].replace('מדינה',str(info2r['citizenship'].iloc[i]) + ', מדינה')


In [169]:
info2r.to_excel("monitoring.xlsx")

In [170]:
mont2report_w_cumsum['clients name'] = mont2report_w_cumsum['clients name'].str.replace('\d+', '')

In [171]:
info2r = info2r.merge(mont2report_w_cumsum[['clients name','cumsum']],how = "left", on = 'clients name')


In [172]:
filepath_check = transction_folder_path+"\*טמפלייט*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    template = textfile
document = MailMerge(template)
print(document.get_merge_fields())

{'company_name', 'person_citizenship', 'Content', 'city', 'person_birth', 'person_id', 'country', 'person_name', 'address', 'Title', 'report_id', 'company_id', 'company_birth', 'company_citizeship'}


In [173]:

for i in info2r.index:
     #template1 = 'test1.docx'
    document = MailMerge(template)
    document.merge(
        person_name = str(info2r['person name'][i]),
        company_name = str(info2r['company name'][i]),
        person_birth = str(info2r['person birth'][i]),
        company_birth = str(info2r['company birth'][i]),
        Title = str(info2r['Title'][i]),
        person_citizenship = str(info2r['person citizenship'][i]),
        Content = str(info2r['Content'][i]),
        country = str(info2r['country'][i]),
        person_id = str(info2r['person id'][i]),
        report_id = str(info2r['report_id'][i]),
        city = str(info2r['city'][i]),
        address = str(info2r['address'][i]),
        company_id = str(info2r['company id'][i]),
        company_citizenship = str(info2r['company citizeship'][i]))
                                            
    

    output =  str(info2r['report_name'][i]) + "-" + str(info2r['report_id'][i]) + '.docx'
    document.write(output)

In [174]:
check[['deal number','check number','bank number','account number','branch number']] = check[['deal number','check number','bank number','account number','branch number']].astype(str)
check_ow[['deal number','check number','bank number','account number','branch number']] = check_ow[['deal number','check number','bank number','account number','branch number']].astype(str)

In [175]:
report_tr = mont2report_w_cumsum[['clients name','cumsum']]

check_min = check[['deal number','date','clients name','check number','bank number','account number','branch number','owner name','nis sum','cumsum']]
report_tr_min = check_min[check_min['cumsum'].isin(report_tr['cumsum'])]

In [176]:
i=0
for_word_table=[]
xlsx=pd.ExcelWriter(transction_folder_path+r"/report.xlsx", engine='openpyxl', mode='w')
#round the sum in ils
report_tr_min["nis sum"]=report_tr_min["nis sum"].round()

#creating a unique list of the sender names
ls_cumsum=report_tr_min['cumsum'].unique().tolist()

#loop each sender transctions in a unique table
for name in ls_cumsum:
    df=report_tr_min[report_tr_min["cumsum"]==name]  
    data = [["סהכ",df["nis sum"].sum()]]
    last_row = pd.DataFrame(data, columns = ["clients name", "nis sum"])
    orderd=pd.concat([df,last_row])
    orderd.to_excel(xlsx, sheet_name='Reports',startrow=i,index=False)
    for_word_table.append(orderd)
    i=i+len(orderd)+2
            
    xlsx.save()

In [177]:
 
check_min_ow = check_ow[['deal number','date','clients name','check number','bank number','account number','branch number','owner name','nis sum','cumsum']]
report_tr_min_ow = check_min_ow[check_min_ow['cumsum'].isin(mont_ow_clients_w_cumsum['cumsum'])]

In [178]:
j=0
for_word_table_ow=[]
xlsx=pd.ExcelWriter(transction_folder_path+r"/report_ow.xlsx", engine='openpyxl', mode='w')
#round the sum in ils
report_tr_min_ow["nis sum"]=report_tr_min_ow["nis sum"].round()

#creating a unique list of the sender names
ls_customer=report_tr_min["clients name"].unique().tolist()
ls_cumsum_ow=mont_ow_clients_w_cumsum['cumsum'].unique().tolist()

#loop each sender transctions in a unique table
for name in ls_cumsum_ow:
    df=report_tr_min_ow[report_tr_min_ow["cumsum"]==name]  
    data = [["סהכ",df["nis sum"].sum()]]
    last_row = pd.DataFrame(data, columns = ["clients name", "nis sum"])
    orderd=pd.concat([df,last_row])
    orderd.to_excel(xlsx, sheet_name='Reports',startrow=j,index=False)
    for_word_table_ow.append(orderd)
    j=j+len(orderd)+2
            
    xlsx.save()

In [179]:
info2r = info2r.set_index('cumsum')


In [180]:
info2r_nona = info2r[info2r.index.notnull()]

In [181]:
for title in ls_cumsum:
    
    data = check[check['cumsum'] == title].round()
    table = [["סהכ",data["nis sum"].sum().round(0)]]
    last_row = pd.DataFrame(table, columns = ["owner name", "nis sum"])
    orderd=pd.concat([data,last_row])
    orderd=orderd[['deal number','date','clients name','check number','bank number','account number','branch number','owner name','nis sum']].fillna(' ')
    
    orderd=change_names_and_order(transction_folder_path+r"/change_names.xlsx",orderd)
    
    orderd['תאריך פעולה'] = orderd['תאריך פעולה'].astype(str)
    orderd['סכום'] = orderd['סכום'].astype(str)
    
    doc = docx.Document(str(info2r_nona['report_name'].loc[title]) + "-" + str(info2r_nona['report_id'].loc[title]) + ".docx")  

    doc.add_page_break()
    t = doc.add_table(orderd.shape[0]+1, orderd.shape[1])
                        
              

    for j in range(orderd.shape[-1]):
        t.cell(0,j).text = orderd.columns[j]

        # add the rest of the data frame
    for i in range(orderd.shape[0]):
        for j in range(orderd.shape[-1]):
            t.cell(i+1,j).text = str(orderd.values[i,j])

    t.style = 'Grid Table 4 Accent 5'
    
    doc.save(str(info2r_nona['report_name'].loc[title]) + "-" + str(info2r_nona['report_id'].loc[title]) + ".docx")


In [182]:
info2r = info2r.reset_index()

In [183]:
mont_ow_clients_w_cumsum['clients name'] = mont_ow_clients_w_cumsum['clients name'].str.replace('\d+','')

In [184]:

info2r_ow = info2r[['report_name','report_id','clients name']].merge(mont_ow_clients_w_cumsum[['clients name','cumsum']], on = 'clients name', how = 'inner')
info2r_ow = info2r_ow.drop_duplicates('cumsum')
info2r_ow = info2r_ow.set_index('cumsum')


for title in ls_cumsum_ow:    
    data = check_ow[check_ow['cumsum'] == title]
    table = [["סהכ",data["nis sum"].sum().round(0)]]
    last_row = pd.DataFrame(table, columns = ["owner name", "nis sum"])
    orderd=pd.concat([data,last_row])
    orderd=orderd[['deal number','date','clients name','check number','bank number','account number','branch number','owner name','nis sum']].fillna(" ")
    orderd=change_names_and_order(transction_folder_path+r"/change_names.xlsx",orderd)
    
    orderd['תאריך פעולה'] = orderd['תאריך פעולה'].astype(str)
    orderd['סכום'] = orderd['סכום'].astype(str)
    
    doc = docx.Document(str(info2r_ow['report_name'].loc[title]) + "-" + str(info2r_ow['report_id'].loc[title]) + ".docx")  

    doc.add_page_break()
    t = doc.add_table(orderd.shape[0]+1, orderd.shape[1])                   
              

    for j in range(orderd.shape[-1]):
        t.cell(0,j).text = orderd.columns[j]

        # add the rest of the data frame
    for i in range(orderd.shape[0]):
        for j in range(orderd.shape[-1]):
            t.cell(i+1,j).text = str(orderd.values[i,j])

    t.style = 'Grid Table 4 Accent 5'
   
    doc.save(str(info2r_ow['report_name'].loc[title]) + "-" + str(info2r_ow['report_id'].loc[title]) + ".docx")
    
       

In [237]:
mont['clients name'] = mont['clients name'].str.replace('\d+', '')
mont = mont.merge(info2r[['clients name','finaltypedown']], on = 'clients name', how = 'left')
mont = mont.drop_duplicates('cumsum')



In [239]:
mont['final status'] =''
for i in mont.index:
    if mont['reported'][i] != 'check':
        mont['final status'][i] = 'דווח בשלושת החודשים האחרונים'
    elif mont['fsp_check'][i] != 'check':
        mont['final status'][i] = 'נש"מ'
    elif (mont['finaltypedown'][i] == 'm'):
        mont['final status'][i] = 'm'
    elif ((mont['nis sum'][i] >= 50000) & (mont['U50'][i] == 0)):
        mont['final status'][i] = 'הועבר דיווח רגיל'
    elif ((mont['סיכון/ללא דיווח רגיל'][i]== 'סיכון') & (mont['U5'][i] == 0)):
        mont['final status'][i] = 'הועבר דיווח רגיל'
    elif ((mont['nis sum'][i] <= 47000) & (mont['סיכון/ללא דיווח רגיל'][i] != 'סיכון')):
        mont['final status'][i] = 'סך עסקאות נמוך מרף הדיווח' 
    elif (mont['nis sum'][i] <= 4700):
        mont['final status'][i] = 'סך עסקאות נמוך מרף הדיווח'
    else:
        mont['final status'][i] = mont['finaltypedown'][i]
        
mont['final status'] = mont['final status'].fillna('אין פעילות במשך שלושה ימים ברציפות')
mont['final status'] = mont['final status'].replace({'pr':'פיצול בסיכון','p':'פיצול','dr':'דירדוס בסיכון','d':'דירדוס','m':'מושך משותף','mr':'מושך משותף בסיכון','pm':'פיצול ומושך משותף'})


mont[['clients name','nis sum','final status']].to_excel('ניטור.xlsx', sheet_name = 'ניטור', index = False)

        